<a href="https://colab.research.google.com/github/tiffanytang34/Age_Detection_Using_Facial_Image_with_CNN/blob/main/3_deep_learning_CNN_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# Imports

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import cv2
import os
from zipfile import ZipFile
import time
from datetime import datetime
import itertools

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, GlobalAveragePooling2D
from tensorflow.keras import utils
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard

# Setting random seeds to reduce the amount of randomness in the neural net weights and results.
# The results may still not be exactly reproducible.
np.random.seed(42)
tf.random.set_seed(42)

In [5]:
# Checking the installed version of TensorFlow.
# The code in this notebook was written using TensorFlow version 2.8.0.

tf.__version__

'2.8.0'

**Ensure that the Runtime Type for this notebook is set to GPU**.

Run the code below to check for GPU. If a GPU device is not found, change the runtime type under *Runtime* &#10141; *Change runtime type* &#10141; *Hardware accelerator* &#10141; *GPU* and run the notebook from the beginning again.

In [6]:
# Testing to ensure GPU is being utilized.

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


## Deep Learning: Data Importing

In [7]:
# Unzipping the dataset file combined_faces.zip

combined_faces_zip_path = "/content/drive/My Drive/Age_Detection/combined_faces.zip"

with ZipFile(combined_faces_zip_path, 'r') as myzip:
    myzip.extractall()
    print('Done unzipping combined_faces.zip')

Done unzipping combined_faces.zip


In [8]:
# Importing the table showing the breakdown of age-ranges into classes.

combined_classes = pd.read_csv("/content/drive/My Drive/Age_Detection/combined_faces_classes_summary.csv")
combined_classes

,Class label,Age-ranges (classes),No. of images,Class balance (%)
0,0,1 - 2,3192,9.53
1,1,3 - 9,2816,8.41
2,2,10 - 20,3136,9.37
3,3,21 - 25,3474,10.37
4,4,26 - 27,3217,9.61
5,5,28 - 31,3063,9.15
6,6,32 - 36,3086,9.22
7,7,37 - 45,3207,9.58
8,8,46 - 54,2802,8.37
9,9,55 - 65,2796,8.35


In [9]:
# Importing the training and testing datasets to create tensors of images using the filename paths.

train_df = pd.read_csv("/content/drive/My Drive/Age_Detection/images_filenames_labels_train.csv")
test_df = pd.read_csv("/content/drive/My Drive/Age_Detection/images_filenames_labels_test.csv")

In [10]:
train_df.head()

,filename,age,target
0,17_21.jpg,17,2
1,9_285.jpg,9,1
2,3_306.jpg,3,1
3,40_314.jpg,40,7
4,21_178.jpg,21,3


In [11]:
test_df.head()

,filename,age,target
0,3_564.jpg,3,1
1,41_114.jpg,41,7
2,39_272.jpg,39,7
3,33_90.jpg,33,6
4,4_269.jpg,4,1


In [12]:
train_df.shape

(23440, 3)

In [13]:
test_df.shape

(10046, 3)